In [41]:
import pandas as pd
import numpy as np

# Charger les données
df = pd.read_csv('data_defi3.csv', sep=';')

# Examiner les données
print("Informations sur le dataset:")
print(df.info())
print("\nStatistiques descriptives:")
print(df.describe())

# Vérifier les valeurs manquantes
print("\nValeurs manquantes:")
print(df.isnull().sum())


Informations sur le dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23514 entries, 0 to 23513
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Libellé.Prescription  23514 non-null  object 
 1   Avis.Pharmaceutique   23141 non-null  object 
 2   PLT                   23514 non-null  float64
dtypes: float64(1), object(2)
memory usage: 551.2+ KB
None

Statistiques descriptives:
                PLT
count  23514.000000
mean       3.903972
std        3.397882
min        1.100000
25%        1.100000
50%        2.200000
75%        6.400000
max       11.000000

Valeurs manquantes:
Libellé.Prescription      0
Avis.Pharmaceutique     373
PLT                       0
dtype: int64


In [42]:
# Examiner la distribution des classes PLT
print("\nDistribution des classes PLT:")
print(df['PLT'].value_counts().sort_index())

# Créer la variable cible binaire
# Classes graves: 4, 5, 6.3, 6.4
classes_graves = [4.0, 4.1, 4.2, 4.3, 5.0, 5.1, 5.2, 5.3, 6.3, 6.4]
df['est_grave'] = df['PLT'].apply(lambda x: 1 if x in classes_graves else 0)

print("\nDistribution de la variable cible:")
print(df['est_grave'].value_counts())
print(f"Proportion d'erreurs graves: {df['est_grave'].mean():.2%}")



Distribution des classes PLT:
PLT
1.1     10082
1.2      1240
1.3       139
2.1        78
2.2       280
2.4       401
3.1       952
3.2        25
4.1      2402
4.2       670
5.1       516
5.2        31
5.3       287
6.1         4
6.2       341
6.3        89
6.4       546
7.0         8
8.1        57
8.2       387
8.3      1059
8.4       374
8.5      1101
9.1        20
10.0      742
11.0     1683
Name: count, dtype: int64

Distribution de la variable cible:
est_grave
0    18973
1     4541
Name: count, dtype: int64
Proportion d'erreurs graves: 19.31%


In [43]:
# Traiter les valeurs manquantes dans 'Avis.Pharmaceutique'
# Option 1: Supprimer les lignes avec commentaires manquants
df_clean = df.dropna(subset=['Avis.Pharmaceutique'])

# Option 2 (alternative): Remplacer par une chaîne vide
# df['Avis.Pharmaceutique'] = df['Avis.Pharmaceutique'].fillna('')

print(f"\nNombre de lignes après nettoyage: {len(df_clean)}")



Nombre de lignes après nettoyage: 23141


In [44]:
import re
from unidecode import unidecode

def nettoyer_texte(texte):
    """
    Nettoie le texte pharmaceutique
    """
    if pd.isna(texte):
        return ""

    # Convertir en minuscules
    texte = texte.lower()

    # Supprimer les dates (format jj/mm/aa)
    texte = re.sub(r'\d{1,2}/\d{1,2}/\d{2,4}', '', texte)

    # Supprimer les ponctuations (virgules, points, etc.)
    texte = re.sub(r'[,.\:;\!\?\-\(\)\[\]\{\}\"\'\/]', ' ', texte)

    # Supprimer les chiffres seuls (garder les dosages comme "3,75mg")
    # texte = re.sub(r'\b\d+\b', '', texte)

    # Supprimer les caractères spéciaux sauf les lettres, chiffres et espaces
    texte = re.sub(r'[^\w\s,.]', ' ', texte)

    # Supprimer les lettres seules (isolées)
    texte = re.sub(r'\b[a-z]\b', '', texte)

    # Supprimer les espaces multiples
    texte = re.sub(r'\s+', ' ', texte).strip()

    return texte

# Appliquer le nettoyage
df_clean['texte_nettoye'] = df_clean['Avis.Pharmaceutique'].apply(nettoyer_texte)

# Afficher quelques exemples
print("\nExemples de textes nettoyés:")
for i in range(5):
    print(f"\nOriginal: {df_clean['Avis.Pharmaceutique'].iloc[i]}")
    print(f"Nettoyé:  {df_clean['texte_nettoye'].iloc[i]}")


Exemples de textes nettoyés:

Original: 30/12/16 pas d'indication 
Nettoyé:  pas indication

Original: 22/12/16 recommandé -> IMOVANE 3,75MG CP, 1 au coucher. EB 
Nettoyé:  recommandé imovane 3 75mg cp 1 au coucher eb

Original: au vue de la DFG, il est recommandé d'administrer 1mg/48h, (données GPR), Veuillez réévaluer la prescription, AB 
Nettoyé:  au vue de la dfg il est recommandé administrer 1mg 48h données gpr veuillez réévaluer la prescription ab

Original: Dose curative et absence d'ATCD gastrique retrouvé, il es recommandé de réduire la posologie à 20 mg/jour   
Nettoyé:  dose curative et absence atcd gastrique retrouvé il es recommandé de réduire la posologie à 20 mg jour

Original: posologie infraT veuillez réévaluer la posologie, nous proposons 30mg/kG, AB 
Nettoyé:  posologie infrat veuillez réévaluer la posologie nous proposons 30mg kg ab


/tmp/ipython-input-1604046701.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['texte_nettoye'] = df_clean['Avis.Pharmaceutique'].apply(nettoyer_texte)


In [46]:
# Longueur moyenne des commentaires
df_clean['longueur_texte'] = df_clean['texte_nettoye'].apply(len)
print(f"\nLongueur moyenne des commentaires: {df_clean['longueur_texte'].mean():.1f} caractères")

# Nombre moyen de mots
df_clean['nb_mots'] = df_clean['texte_nettoye'].apply(lambda x: len(x.split()))
print(f"Nombre moyen de mots: {df_clean['nb_mots'].mean():.1f} mots")

# Comparer pour les cas graves vs non-graves
print("\nStatistiques par gravité:")
print(df_clean.groupby('est_grave')[['longueur_texte', 'nb_mots']].mean())


Longueur moyenne des commentaires: 72.9 caractères
Nombre moyen de mots: 12.8 mots

Statistiques par gravité:
           longueur_texte    nb_mots
est_grave                           
0               71.690593  12.686080
1               77.845406  13.363074


/tmp/ipython-input-3480693702.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['longueur_texte'] = df_clean['texte_nettoye'].apply(len)
/tmp/ipython-input-3480693702.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['nb_mots'] = df_clean['texte_nettoye'].apply(lambda x: len(x.split()))


In [40]:
from sklearn.model_selection import train_test_split

# Séparer les features (X) et la cible (y)
X = df_clean['texte_nettoye']
y = df_clean['est_grave']

# Division train/test (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y  # Préserver la proportion des classes
)

print(f"\nTaille du jeu d'entraînement: {len(X_train)}")
print(f"Taille du jeu de test: {len(X_test)}")
print(f"\nDistribution dans le train: {y_train.value_counts(normalize=True)}")
print(f"Distribution dans le test: {y_test.value_counts(normalize=True)}")



Taille du jeu d'entraînement: 18512
Taille du jeu de test: 4629

Distribution dans le train: est_grave
0    0.804343
1    0.195657
Name: proportion, dtype: float64
Distribution dans le test: est_grave
0    0.804277
1    0.195723
Name: proportion, dtype: float64


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Créer le vectoriseur TF-IDF
tfidf = TfidfVectorizer(
    max_features=5000,      # Garder les 5000 mots les plus importants
    ngram_range=(1, 2),     # Unigrammes et bigrammes
    min_df=5,               # Ignorer les mots apparaissant dans < 5 documents
    max_df=0.8,             # Ignorer les mots trop fréquents (> 80%)
    strip_accents='unicode'
)

# Transformer les données
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

print(f"\nDimensions de la matrice TF-IDF train: {X_train_tfidf.shape}")
print(f"Dimensions de la matrice TF-IDF test: {X_test_tfidf.shape}")



Dimensions de la matrice TF-IDF train: (18512, 5000)
Dimensions de la matrice TF-IDF test: (4629, 5000)


In [47]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# Créer et entraîner le modèle
lr_model = LogisticRegression(
    max_iter=1000,
    class_weight='balanced',  # Important pour les classes déséquilibrées
    random_state=42
)

lr_model.fit(X_train_tfidf, y_train)

# Prédictions
y_pred_lr = lr_model.predict(X_test_tfidf)
y_pred_proba_lr = lr_model.predict_proba(X_test_tfidf)[:, 1]

# Évaluation
print("\n" + "="*60)
print("RÉGRESSION LOGISTIQUE - RÉSULTATS")
print("="*60)
print("\nRapport de classification:")
print(classification_report(y_test, y_pred_lr, target_names=['Non-Grave', 'Grave']))

print("\nMatrice de confusion:")
print(confusion_matrix(y_test, y_pred_lr))

print(f"\nROC-AUC Score: {roc_auc_score(y_test, y_pred_proba_lr):.4f}")



RÉGRESSION LOGISTIQUE - RÉSULTATS

Rapport de classification:
              precision    recall  f1-score   support

   Non-Grave       0.97      0.90      0.94      3723
       Grave       0.69      0.89      0.78       906

    accuracy                           0.90      4629
   macro avg       0.83      0.90      0.86      4629
weighted avg       0.92      0.90      0.90      4629


Matrice de confusion:
[[3360  363]
 [ 100  806]]

ROC-AUC Score: 0.9662


In [48]:
from xgboost import XGBClassifier

# Calculer le ratio de déséquilibre
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()

# Créer et entraîner le modèle
xgb_model = XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    scale_pos_weight=scale_pos_weight,  # Gérer le déséquilibre
    random_state=42,
    eval_metric='logloss'
)

xgb_model.fit(X_train_tfidf, y_train)

# Prédictions
y_pred_xgb = xgb_model.predict(X_test_tfidf)
y_pred_proba_xgb = xgb_model.predict_proba(X_test_tfidf)[:, 1]

# Évaluation
print("\n" + "="*60)
print("XGBOOST - RÉSULTATS")
print("="*60)
print("\nRapport de classification:")
print(classification_report(y_test, y_pred_xgb, target_names=['Non-Grave', 'Grave']))

print(f"\nROC-AUC Score: {roc_auc_score(y_test, y_pred_proba_xgb):.4f}")



XGBOOST - RÉSULTATS

Rapport de classification:
              precision    recall  f1-score   support

   Non-Grave       0.97      0.92      0.94      3723
       Grave       0.72      0.87      0.79       906

    accuracy                           0.91      4629
   macro avg       0.85      0.89      0.87      4629
weighted avg       0.92      0.91      0.91      4629


ROC-AUC Score: 0.9576


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Créer et entraîner le modèle
rf_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=20,
    min_samples_split=10,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1  # Utiliser tous les CPU
)

rf_model.fit(X_train_tfidf, y_train)

# Prédictions
y_pred_rf = rf_model.predict(X_test_tfidf)
y_pred_proba_rf = rf_model.predict_proba(X_test_tfidf)[:, 1]

# Évaluation
print("\n" + "="*60)
print("RANDOM FOREST - RÉSULTATS")
print("="*60)
print("\nRapport de classification:")
print(classification_report(y_test, y_pred_rf, target_names=['Non-Grave', 'Grave']))

print(f"\nROC-AUC Score: {roc_auc_score(y_test, y_pred_proba_rf):.4f}")



RANDOM FOREST - RÉSULTATS

Rapport de classification:
              precision    recall  f1-score   support

   Non-Grave       0.95      0.89      0.92      3723
       Grave       0.65      0.81      0.72       906

    accuracy                           0.88      4629
   macro avg       0.80      0.85      0.82      4629
weighted avg       0.89      0.88      0.88      4629


ROC-AUC Score: 0.9286


In [49]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold

# Exemple avec la Régression Logistique
param_grid = {
    'C': [0.1, 1, 10, 100],  # Force de régularisation
    'penalty': ['l1', 'l2'],  # Type de régularisation
    'solver': ['liblinear', 'saga']
}

# Cross-validation stratifiée
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Grid Search
grid_search = GridSearchCV(
    LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42),
    param_grid,
    cv=cv,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train_tfidf, y_train)

print("\nMeilleurs paramètres:", grid_search.best_params_)
print(f"Meilleur score (ROC-AUC): {grid_search.best_score_:.4f}")

# Utiliser le meilleur modèle
best_model = grid_search.best_estimator_


Fitting 5 folds for each of 16 candidates, totalling 80 fits

Meilleurs paramètres: {'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}
Meilleur score (ROC-AUC): 0.9561


In [50]:
from sklearn.model_selection import cross_val_score

# Validation croisée avec 5 folds
cv_scores = cross_val_score(
    best_model,
    X_train_tfidf,
    y_train,
    cv=5,
    scoring='roc_auc'
)

print(f"\nScores de validation croisée (ROC-AUC):")
print(f"Scores: {cv_scores}")
print(f"Moyenne: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")



Scores de validation croisée (ROC-AUC):
Scores: [0.95597879 0.96089877 0.95470815 0.95556735 0.95485077]
Moyenne: 0.9564 (+/- 0.0046)


In [51]:
# Pour la régression logistique
feature_names = tfidf.get_feature_names_out()
coefficients = best_model.coef_[0]

# Top 20 mots associés aux erreurs GRAVES
top_grave = np.argsort(coefficients)[-20:]
print("\nTop 20 mots associés aux erreurs GRAVES:")
for idx in reversed(top_grave):
    print(f"  {feature_names[idx]}: {coefficients[idx]:.4f}")

# Top 20 mots associés aux erreurs NON-GRAVES
top_non_grave = np.argsort(coefficients)[:20]
print("\nTop 20 mots associés aux erreurs NON-GRAVES:")
for idx in top_non_grave:
    print(f"  {feature_names[idx]}: {coefficients[idx]:.4f}")



Top 20 mots associés aux erreurs GRAVES:
  max: 9.9582
  doublon: 5.7833
  indication: 5.4399
  deja: 5.1683
  suspendre: 5.1462
  diminuer: 5.0312
  maximum: 4.6034
  arret: 4.4824
  redondance: 4.3330
  double: 4.1895
  maximale: 3.9469
  supprimer: 3.9271
  arreter: 3.7833
  association: 3.6548
  stop: 3.3527
  risque: 3.2993
  lignes: 3.1262
  avec: 3.0996
  stopper: 2.9593
  surdosage: 2.9526

Top 20 mots associés aux erreurs NON-GRAVES:
  substituer: -3.4492
  substituer par: -3.4380
  hus: -3.2324
  avons: -3.0274
  substitue: -2.9639
  substitue par: -2.9491
  lp: -2.5820
  aux: -2.4993
  par: -2.4977
  prescrire: -2.4370
  aux hus: -2.3974
  pendant: -2.3950
  prevoir: -2.3340
  surveiller: -2.2325
  preciser: -2.1103
  gelules: -2.1098
  substitution: -2.0483
  comprimes: -2.0277
  vie: -1.9840
  hospitalisation: -1.9786


In [52]:
# Créer un DataFrame avec les prédictions
results_df = pd.DataFrame({
    'texte': X_test,
    'vrai_label': y_test,
    'prediction': y_pred_lr,
    'probabilite': y_pred_proba_lr
})

# Faux positifs (prédit grave mais ne l'est pas)
faux_positifs = results_df[(results_df['vrai_label'] == 0) & (results_df['prediction'] == 1)]
print("\nExemples de FAUX POSITIFS:")
print(faux_positifs.head(10))

# Faux négatifs (prédit non-grave mais l'est)
faux_negatifs = results_df[(results_df['vrai_label'] == 1) & (results_df['prediction'] == 0)]
print("\nExemples de FAUX NÉGATIFS:")
print(faux_negatifs.head(10))



Exemples de FAUX POSITIFS:
                                                   texte  vrai_label  \
10903  posologie à revoir prescrit 40ml soit 400mg de...           0   
11763                     arrondir à 350mg 3 jour cécile           0   
6170        refusé pour le motif suivant ne pas préparer           0   
10319  dose curative 100 ui anti xa kg 2 soit pour un...           0   
12526                         attention date de début eb           0   
17787  la prudence est recommandée chez les patients ...           0   
18391  non compatible avec cancidas et nutrition pare...           0   
1989        habituellement 15mg kg jour soit 260 mg jour           0   
620    au vu de âge et de la fragilité du patient ben...           0   
22093  patient avec fonction rénale normale proposons...           0   

       prediction  probabilite  
10903           1     0.702136  
11763           1     0.636549  
6170            1     0.803856  
10319           1     0.512808  
12526         

In [54]:
from sklearn.metrics import precision_recall_curve, f1_score

# Courbe précision-recall
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba_lr)

# Trouver le seuil optimal pour maximiser le F1-score
f1_scores = 2 * (precision * recall) / (precision + recall + 1e-10)
optimal_idx = np.argmax(f1_scores)
optimal_threshold = thresholds[optimal_idx]

print(f"\nSeuil optimal: {optimal_threshold:.3f}")
print(f"F1-score au seuil optimal: {f1_scores[optimal_idx]:.4f}")
print(f"Précision: {precision[optimal_idx]:.4f}")
print(f"Rappel: {recall[optimal_idx]:.4f}")

# Appliquer le seuil optimal
y_pred_optimal = (y_pred_proba_lr >= optimal_threshold).astype(int)

print("\nPerformances avec le seuil optimal:")
print(classification_report(y_test, y_pred_optimal, target_names=['Non-Grave', 'Grave']))



Seuil optimal: 0.681
F1-score au seuil optimal: 0.8210
Précision: 0.8491
Rappel: 0.7947

Performances avec le seuil optimal:
              precision    recall  f1-score   support

   Non-Grave       0.95      0.97      0.96      3723
       Grave       0.85      0.79      0.82       906

    accuracy                           0.93      4629
   macro avg       0.90      0.88      0.89      4629
weighted avg       0.93      0.93      0.93      4629



In [55]:
import joblib

# Sauvegarder le modèle et le vectoriseur
joblib.dump(best_model, 'modele_classification_ip.pkl')
joblib.dump(tfidf, 'vectoriseur_tfidf.pkl')

print("\nModèle et vectoriseur sauvegardés!")

# Pour réutiliser plus tard:
# modele_charge = joblib.load('modele_classification_ip.pkl')
# tfidf_charge = joblib.load('vectoriseur_tfidf.pkl')



Modèle et vectoriseur sauvegardés!
